In [ ]:
## `[Preposition]`

#### 1. 10월 한달간 구매하는 고객을 고객의 특성을 활용해 예측할 수 있다

​

## `[Scenario]`

#### 1. 10월 10일까지 구매한 데이터를 바탕으로 학습을 진행

#### 2. 10월 10~30일 사이에 구매한 결과를 바탕으로 모델 평가

#### 3. 만약 실효성 있는 결과가 나올 경우, 구매 고객을 예측할 때 보조지표로 활용할 수 있다는 것을 의미

​

## `사용 데이터`

​

### `1. 1001시점_online_모델용_191001.csv (from Practice.ipynb)`

​

##### 1) 관수 선임님 폴더에 있는 "1001시점_online_모델용_191001.csv",즉 멤버별 온라인 구매 패턴 데이터를 활용

​

##### 2) 포함 칼럼 :'MBR_NO', 'Y1_총구매일수', 'Y1_총구매건수', 'Y1_총구매금액', '최종구매경과기간(일)', '회원가입경과기간(월)', '구매주기평균', '단기구매주기여부', '3M_WEB_CNT', '2M_WEB_CNT', '1M_WEB_CNT', '3W_WEB_CNT', '2W_WEB_CNT', '1WK_WEB_CNT', 'SMS_RECPTN_AGR_YN'

​

##### 3) 데이터 사이즈 : 192792, 15

​

##### 4) 데이터 전처리 : 단기구매주기 여부를 0/1 encoding하고, WEB_CNT(주기별 인터넷 로그 카운트)가 Null인 경우 0으로 fill_na 해줌

​

====================================================================

​

### `2. online/ONLINE_ORDER_YEAR.parquet (from Practice.ipynb)`

​

##### 1) 온라인 구매 내역을 담은 파케이 형식 데이터로, 현재 시점 이전의 주문 데이터를 포함하고 있음

​

##### 2) 포함 칼럼 : 'MBR_NO', 'ORD_DT', 'ORD_NO', 'GOD_NO', 'SAP_GOD_NO', 'FINAL_ORD_QTY', 'ORD_AMT'

​

##### 3) 데이터 사이즈 : 1583776, 7

​

##### 4) 데이터 전처리 : 1번 데이터와 멤버코드를 기준으로 병합하여, 멤버 별 구매 내역을 DataFrame으로 만듦 -> 주문일자 데이터를 수치타입으로 변경 -> target_value 칼럼을 추가해 10월 이후의 주문 내역에 1의 값을 추가함

​

​

### ` 3. online/DW_MBR (from Online_analysis.ipynb)`

​

##### 1) 온라인 멤버의 정보를 담은 데이터로, 멤버십 정보와 성별/생년월일 등의 인구통계적 정보를 포함하고 있음. 이 중 수신 동의 고객만 남기고, 중복 고객을 제거하여 40만6천을 남김

​

##### 2) 포함 칼럼 : 'MBR_NO', 'MBR_TP_CD', 'MBR_ATRB_CD', 'MBR_BRTHDY', 'MBR_SEX_SECT_CD', 'JOIN_DT', 'SMS_RECPTN_AGR_YN', 'CRM_GRD_CD', 'ONLNE_GRD_CD' 

​

##### 3) 데이터 사이즈 : 406242, 9

​

​

##### 4) 데이터 스키마 : MBR_NO -> ['MB000000000000000'] / MBR_TP_CD -> ['UNITY_MBR' 'ONLINE_MBR'] / MBR_ATRB_CD -> ['GNRL_MBR' 'CHEIL_FSHN' 'CHEIL_FSHN_RTL' 'GRPCO' 'CHEIL'] / MBR_BRTHDY -> '19270801' / MBR_SEX_SECT_CD -> ['FEMALE' 'MALE' None]

##### JOIN_DT -> ['2000-00-00T00:00:00.000000000'] / SMS_RECPTN_AGR_YN -> ['Y'] / CRM_GRD_CD -> ['FAM' None 'GLD' 'SLVR' 'DRMC' 'PLTN' 'LTNCY'] / ONLNE_GRD_CD -> ['VIP' None] 

​

​

#### 5) 데이터 전처리

##### 1] MBR_TP_CD : 'UNITY_MBR'은 336426, 'ONLINE_MBR'은 69816으로 상대적 소수인 ONLINE_MBR을 1로 encoding

##### 2] MBR_ATRB_CD : 'GNRL_MBR'이 400227, 'GRPCO'은 5051, 나머지는 200대. 인코딩하지 않고 유지함.

##### 3] BRTHDY_MNTH_YN : 9/10월에 생일이 들어있는 여부를 encoding함. yes를 1로 encode.

##### 4] MBR_AGE : 생년월일로부터 나이를 추출

##### 5] MBR_SEX_SECT_CD : 남자는 0, 여자는 1. na값은 대다수인 여성으로 채워줌. 

##### 6] ONLNE_GRD_CD : VIP를 1, 그렇지 않으면 0으로 encode

##### 7] CRM_GRD_CD : CRM 등급은 그냥 둠. (None값 포함되어 있음)

##### -> 이 결과를 one_hot 하여 새로운 DF를 만들고, 이를 "Online_complete.csv"로 저장

​

​

​

​

# 개선점

​

## 연령 코드가 제대로 binning 되어 있지 않음.

​

## CRM_GRD_CD 등의 범주형 변수는 목표값과의 차이를 살펴 통합하는 방식으로 차원 축소

​

## 목표값의 재정의가 필요함. -> 전환율 ? 10월 이후 구매를 했는지 여부. 즉, target_value

​

## 기존 모델에 target_value를 사용해버림. 이는 모델이 cheating할 수 있다는 것을 의미.

​

## 

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler


import os
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
import itertools as IT
import glob
import datetime as dt
import time
import sys


sys.path.insert(0, '/data/aithe/data/customercube/code/common/')
# import common_path
from dateutil.relativedelta import relativedelta

from xgboost import plot_importance
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV   #Perforing grid search

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import xgboost

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',500)
pd.set_option('max_info_columns',500)
pd.set_option('display.float_format', lambda x : '%.3f' % x )

day_test_end="20191010"


from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD


import datetime as dt
today = dt.datetime.now().strftime('%Y%m%d')

%%time
import common_path_MH

print(os.getcwd())


data=pd.read_csv("hypo0_prepro_comp.csv")

data

data.drop("Unnamed: 0",axis=1,inplace=True)

data_index=data["MBR_NO"]

data=data.set_index("MBR_NO")



# 스케일링 후 다시 칼럼 네임 붙이기
colnames=data.columns

transformer = RobustScaler().fit(data)
data=transformer.transform(data)


data=pd.DataFrame(data)

data.columns=colnames

# 타겟의 샘플 수
target_sample=data.groupby('target_value').count().iloc[1,1]
print(target_sample,"and",data.groupby('target_value').count().iloc[0,1])




print('NON RETURN', round(data['target_value'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('RETURN', round(data['target_value'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

X = data.drop('target_value', axis=1)
y = data['target_value']

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels

# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

# Random Under-Sampling

new_df.to_csv("hypo0_1.csv",encoding="CP949")

data = data.sample(frac=1)

# amount of fraud classes 492 rows.
Return = data.loc[data['target_value'] == 1]
Non_Return = data.loc[data['target_value'] == 0][:target_sample]

normal_distributed_df = pd.concat([Return, Non_Return])

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

new_df.head()

print('Distribution of the Classes in the subsample dataset')
print(new_df['target_value'].value_counts()/len(new_df))

colors = ["#0101DF", "#DF0101"]


sns.countplot('target_value', data=new_df, palette=colors)
plt.title('Equally Distributed Classes', fontsize=14)
plt.show()


f, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,20))

# Entire DataFrame
corr = data.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)
ax1.set_title("Imbalanced Correlation Matrix \n (don't use for reference)", fontsize=14)


sub_sample_corr = new_df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2)
ax2.set_title('SubSample Correlation Matrix \n (use for reference)', fontsize=14)
plt.show()

# New_df is from the random undersample data (fewer instances)
X_us = new_df.drop('target_value', axis=1)
y_us = new_df['target_value']



from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(X_us, y_us, test_size=0.2, random_state=42)

# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# Let's implement simple classifiers

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}


# Wow our scores are getting even high scores even when applying cross validation.
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 4) * 100, "% accuracy score")
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD


# Use GridSearchCV to find the best parameters.
from sklearn.model_selection import GridSearchCV
import xgboost

best_xgb_model = xgboost.XGBClassifier(colsample_bytree=0.4,
                 gamma=0.3,                 
                 learning_rate=0.1,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=1e-05,
                 reg_lambda=0.01,
                 subsample=0.95,
                 )

best_xgb_model.fit(X_train,y_train)

training_score = cross_val_score(best_xgb_model, X_train, y_train, cv=5)

# 가장 우수한 성능을 보이는 SVC

model_SVC=SVC()
model_SVC.fit(X=X_train, y=y_train)



y_pred_SVC = model_SVC.predict(X_test)




# 선형 커널 만들어보기
from sklearn.svm import LinearSVC
model_LSVC = LinearSVC()
model_LSVC.fit(X_train,y_train)

# proba = model_LSVC.decision_function(X_train)
# prob = (proba - proba.min()) / (proba.max() - proba.min())
# print('Prob[0]: %.3f' % (1-prob[0]))
# print('Prob[1]: %.3f' % (prob[0]))

cross_val_score(model_LSVC, X_train, y_train, cv=5)



# Logistic Regression model

model_LR =LogisticRegression()
model_LR.fit(X_train, y_train)


y_pred_LR = model_LR.predict(X_test)


model_LR.coef_

model_RF =RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
model_RF.fit(X_train, y_train)


# Random Forest model

y_pred_RF = model_RF.predict(X_test)


for name, importance in zip(X_train, model_RF.feature_importances_):
    print(name, "=", importance)

X_train1=pd.DataFrame(X_train)

# RF 모델의 피처 중요도 플롯

feat_importances = pd.Series(model_RF.feature_importances_, index=X_train1.columns)
feat_importances.nlargest(15).plot(kind='barh')

y_pred = best_xgb_model.predict(X_test)


X_train

y_pred2 = best_xgb_model.predict(X.values)


y_pred2 = model_LSVC.predict(X)


from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y, y_pred2)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()



import sklearn.metrics

print(sklearn.metrics.recall_score(y,y_pred2))
print(sklearn.metrics.precision_score(y,y_pred2))

print(sklearn.metrics.accuracy_score(y,y_pred2))
print(sklearn.metrics.f1_score(y,y_pred2))
print(sklearn.metrics.precision_recall_curve(y,y_pred2))

# Feature importance

# 피처 중요도를 살펴본 결과, 최종구매경과기간(일) 특성이 가장 중요한 것으로 드러남

from matplotlib import pyplot as plt
from sklearn import svm

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

f_importances(model_LSVC.coef_[0], list(X.columns))


# AUROC 구하기

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

random_state = np.random.RandomState(0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=random_state))

y_score = classifier.fit(X_train, y_train).decision_function(X_test)



y_score2 = model_LSVC.fit(X=X_train, y=y_train).decision_function(X_test)

n_classes=len(y_score2)



# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_score2)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score2.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

pd.DataFrame(fpr).T

fpr[2]

tpr[2]

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()













# 10월 11~30일 데이터를 기존 모델에 넣어 맞춰보기 

# 10월 11~30일 사이의 구매 고객을 정리해야함
# 이를 위해, 데이터의 기간을 설정해 target_value 달아주고
# X, Y를 분리시킨 다음
# X를 투입해 결과값을 pred에 저장하고,
# Y와 대치하여 confusion matrix 만들기




data2=pd.read_csv("hypo0_prepro_comp_1010.csv")

data2

data2.drop("Unnamed: 0",axis=1,inplace=True)

data2=data2.set_index("MBR_NO")

# 스케일링 후 다시 칼럼 네임 붙이기
colnames2=data2.columns

transformer2 = RobustScaler().fit(data2)
data2=transformer2.transform(data2)


data2=pd.DataFrame(data2)

data2.columns=colnames

# 타겟의 샘플 수
target_sample2=data2.groupby('target_value').count().iloc[1,1]
print(target_sample2,"and",data2.groupby('target_value').count().iloc[0,1])



print('NON RETURN', round(data2['target_value'].value_counts()[0]/len(data2) * 100,2), '% of the dataset')
print('RETURN', round(data2['target_value'].value_counts()[1]/len(data2) * 100,2), '% of the dataset')

X2 = data2.drop('target_value', axis=1)
y2 = data2['target_value']

sss2 = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss2.split(X2, y2):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X2.iloc[train_index], X2.iloc[test_index]
    original_ytrain, original_ytest = y2.iloc[train_index], y2.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels
# Turn into an array
original_Xtrain2 = original_Xtrain.values
original_Xtest2 = original_Xtest.values
original_ytrain2 = original_ytrain.values
original_ytest2 = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain2, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest2, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain2))
print(test_counts_label/ len(original_ytest2))

y_pred2 = model_RF.predict(X2)

y_pred2 = model_LR.predict(X2)


y_pred2 = model_SVC.predict(X2)

y_pred2 = model_LSVC.predict(X2)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y2, y_pred2)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()



import sklearn.metrics

print(sklearn.metrics.recall_score(y2,y_pred2))
print(sklearn.metrics.precision_score(y2,y_pred2))

print(sklearn.metrics.accuracy_score(y2,y_pred2))
print(sklearn.metrics.f1_score(y2,y_pred2))
print(sklearn.metrics.precision_recall_curve(y2,y_pred2))



# Feature importance

# 피처 중요도를 살펴본 결과, 최종구매경과기간(일) 특성이 가장 중요한 것으로 드러남

from matplotlib import pyplot as plt
from sklearn import svm

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

f_importances(model_LSVC.coef_[0], list(X2.columns))







# AUROC 구하기

# This is explicitly used for undersampling.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Turn the values into an array for feeding the classification algorithms.
X_train2 = X_train2.values
X_test2 = X_test2.values
y_train2 = y_train2.values
y_test2 = y_test2.values





random_state = np.random.RandomState(0)

y_score2 = model_LSVC.fit(X=X_train2, y=y_train2).decision_function(X_test2)

n_classes=len(y_score2)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test2, y_score2)
    roc_auc[i] = auc(fpr[i], tpr[i])
    
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test2.ravel(), y_score2.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])







plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()













import imblearn

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV


print('Length of X (train): {} | Length of y (train): {}'.format(len(original_Xtrain), len(original_ytrain)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(original_Xtest), len(original_ytest)))

# List to append the score and then find the average
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []

# Classifier with optimal parameters
# log_reg_sm = grid_log_reg.best_estimator_
log_reg_sm = LogisticRegression()




rand_log_reg = RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)


# Implementing SMOTE Technique 
# Cross Validating the right way
# Parameters
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
for train, test in sss.split(original_Xtrain, original_ytrain):
    pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    model = pipeline.fit(original_Xtrain[train], original_ytrain[train])
    best_est = rand_log_reg.best_estimator_
    prediction = best_est.predict(original_Xtrain[test])
    
    accuracy_lst.append(pipeline.score(original_Xtrain[test], original_ytrain[test]))
    precision_lst.append(precision_score(original_ytrain[test], prediction))
    recall_lst.append(recall_score(original_ytrain[test], prediction))
    f1_lst.append(f1_score(original_ytrain[test], prediction))
    auc_lst.append(roc_auc_score(original_ytrain[test], prediction))
    
print('---' * 45)
print('')
print("accuracy: {}".format(np.mean(accuracy_lst)))
print("precision: {}".format(np.mean(precision_lst)))
print("recall: {}".format(np.mean(recall_lst)))
print("f1: {}".format(np.mean(f1_lst)))
print('---' * 45)

labels = ['No Fraud', 'Fraud']
smote_prediction = best_est.predict(original_Xtest)
print(classification_report(original_ytest, smote_prediction, target_names=labels))

n_classes=len(y_score)